In [348]:
from openai import OpenAI
import os
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import json
import time

# Weather assistant using the openai assistants API

In [349]:
def get_weather_by_location(latitude, longitude, weather_variable="temperature_2m"):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    # Prepare the API call
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": weather_variable
    }

    # Call the API
    responses = openmeteo.weather_api(url, params=params)

    # Process the first location response
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process hourly data
    hourly = response.Hourly()
    hourly_data_values = hourly.Variables(0).ValuesAsNumpy()

    hourly_data = {
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s"),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s"),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        )
    }
    hourly_data[weather_variable] = hourly_data_values

    hourly_dataframe = pd.DataFrame(data=hourly_data)
    return hourly_dataframe

In [350]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [351]:
assistant = client.beta.assistants.create(
    name="Weather Assistant",
    instructions="You are a friendly assistant that tells me the weather in a location",
    tools=[{
            "type": "function",
            "function": {
                "name": "get_weather_by_location",
                "description": "Get the weather in location based on latitude and longitude",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "latitude": {"type": "string", "description": "The latitude of the location (e.g., 37.7749)"},
                        "longitude": {"type": "string", "description": "The longitude of the location (e.g., -122.4194)"},
                    },
                },
                "required": ["latitude", "longitude"]
                
            }
        }
    ],
    model="gpt-4-1106-preview"
)

In [352]:
thread = client.beta.threads.create()

In [353]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is the weather in San Francisco?"
)

In [354]:
message = client.beta.threads.messages.retrieve(
    thread_id=thread.id,
    message_id=message.id,
)

In [355]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [356]:
# print(run.model_dump_json(indent=2))

In [357]:
# print(thread.model_dump_json(indent=2))

In [358]:
# Wait 5 seconds for the run to complete
time.sleep(10)

In [359]:
retrieved_run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id,
)

In [360]:
print(retrieved_run.model_dump_json(indent=2))

{
  "id": "run_LNlQQzzoVNCzAf7CnIG4T9WN",
  "assistant_id": "asst_EwO5EIKf58leWJ2CAOipgfXM",
  "cancelled_at": null,
  "completed_at": null,
  "created_at": 1700589287,
  "expires_at": 1700589887,
  "failed_at": null,
  "file_ids": [],
  "instructions": "You are a friendly assistant that tells me the weather in a location",
  "last_error": null,
  "metadata": {},
  "model": "gpt-4-1106-preview",
  "object": "thread.run",
  "required_action": {
    "submit_tool_outputs": {
      "tool_calls": [
        {
          "id": "call_B92RWYRP3qs5BShcx74fSSOJ",
          "function": {
            "arguments": "{\"latitude\":\"37.7749\",\"longitude\":\"-122.4194\"}",
            "name": "get_weather_by_location"
          },
          "type": "function"
        }
      ]
    },
    "type": "submit_tool_outputs"
  },
  "started_at": 1700589287,
  "status": "requires_action",
  "thread_id": "thread_HFiHanaX0EDD6yHR9FNVHmeX",
  "tools": [
    {
      "function": {
        "name": "get_weather_by_loc

In [361]:
# EXAMPLE OUTPUT:
# 
#  "required_action": {
#     "submit_tool_outputs": {
#       "tool_calls": [
#         {
#           "id": "call_OjFGC2nqulybgJSlyoRWDk3L",
#           "function": {
#             "arguments": "{\"latitude\":\"51.5074\",\"longitude\":\"-0.1278\"}",
#             "name": "get_weather_by_location"
#           },
#           "type": "function"
#         }
#       ]
#     },
#     "type": "submit_tool_outputs"
#   },

# Check if the run is completed then call the API with the tool outputs
if retrieved_run and retrieved_run.required_action and retrieved_run.required_action.submit_tool_outputs:
    tool_call = retrieved_run.required_action.submit_tool_outputs.tool_calls[0]
    tool_call_id = tool_call.id
    tool_call_function = tool_call.function
    tool_call_arguments = tool_call_function.arguments
    tool_call_arguments_json = json.loads(tool_call_arguments)
    tool_call_function_name = tool_call_function.name

    # Call the API
    weather_data = get_weather_by_location(
        latitude=tool_call_arguments_json["latitude"],
        longitude=tool_call_arguments_json["longitude"],
        weather_variable="temperature_2m"
    )

Coordinates 37.763282775878906°E -122.41285705566406°N
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [362]:
# convert the dataframe to a JSON string
weather_data_json = weather_data.to_json(orient="records")

In [363]:
# convert json to one string:
weather_data_json = "".join(weather_data_json.splitlines())

In [364]:
# Create a new message to the assistant with the weather data and ask for a concise summary
run = client.beta.threads.runs.submit_tool_outputs(
  thread_id=thread.id,
  run_id=run.id,
  tool_outputs=[
    {
      "tool_call_id": tool_call.id,
      "output": weather_data_json + "\n\nPlease summarize the weather in one or two sentences."
    }
  ]
)


In [367]:
messages = client.beta.threads.messages.list(
      thread_id=thread.id
)

for thread_message in messages.data:
    # Iterate over the 'content' attribute of the ThreadMessage, which is a list
    for content_item in thread_message.content:
        # Assuming content_item is a MessageContentText object with a 'text' attribute
        # and that 'text' has a 'value' attribute, print it
        print(content_item.text.value)

The weather in San Francisco shows a pattern of gradual warming throughout the day with temperatures ranging from the lower to upper teens in degrees Celsius. Nighttime temperatures dip slightly, while daytime temperatures peak, likely reaching the high teens or low twenties in degrees Celsius.
What is the weather in San Francisco?


In [366]:
# print(thread_messages.model_dump_json(indent=2))